In [1]:
import os
import sys
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
from os import path
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

#import libraries
from pyecharts.charts import Bar, Line
from pyecharts import options as opts
from pyecharts.globals import ThemeType

from helper_functions import *
from analytic_functions import *

pd.set_option('max_colwidth', None)

In [2]:
total_messages, total_messages_with_id, participant_names = get_messages(messageloc = 'facebookdata/messages/inbox/s_ibkzdafrna/')
len(total_messages)

136076

## GENERAL STATS

In [83]:
most_reacted_to_message(total_messages_with_id)

{'sender_name': 'Anirudh Kuchibhatla',
 'timestamp_ms': 1638805933391,
 'content': "Good news guys, I committed to Columbia's deferred MBA program, which means I will be doing an MBA there in 2-5 years",
 'type': 'Generic',
 'is_unsent': False,
 'reats_better': {'heart': 17}}

In [3]:
### who has sent the most messages
d1 = message_count(total_messages, participant_names)

### most character count
d2 = total_character_count(total_messages, participant_names)

### average character count
d3 = average_character_count(total_messages, participant_names)

## most reacted to message
df = pd.Series(most_reacted_to_message(total_messages_with_id)).to_frame()
render_mpl_table(df.reset_index(), col_width=9.0)
plt.savefig(f'pictures/most_reacted_message')
plt.close()

# ### recieved reactions for every participant
# render_mpl_table(reactions_recieved_by_all(participant_names, total_messages, returntype='pd').reset_index())
# plt.savefig('pictures/reactions_recieved_by_all.png')
# plt.close()

# ### specific react to messages reation
# react_stats = reactions_recieved_by_all(participant_names, total_messages, returntype='dict')

# ### who has reacted what the most
# render_mpl_table(reactions_given_by_all(total_messages).reset_index())
# plt.savefig(f'pictures/reactions_given_by_all')
# plt.close()

# ### group name changes
# d = group_name_changes_timeline(total_messages)

# ### photos count
# d = most_common_photo_sender(total_messages)

# ### groups most common words
# # most_common_words([i['content'] for i in total_messages if 'content' in i]).head(20)

# ### most common tags per person
# tag_correlation_matrix(participant_names, total_messages)
# plt.close()

# ### reply matrix
# gen_reply_matrix(total_messages_with_id, participant_names, False)
# plt.close()

In [46]:
df = pd.DataFrame(pd.Series(d1))
df.columns = ['message_count']
df['average_character_count'] = df.index.map(d3)
df['average_character_count'] = df['average_character_count'].map(lambda x: np.around(x, decimals = 1))
# df

In [47]:
bar = (
 Bar(init_opts=opts.InitOpts(theme=ThemeType.DARK))
 .add_xaxis(list(df.index))
 .add_yaxis("Message Count", list(df.message_count), yaxis_index = 0)
 .add_yaxis("Average Character Count", list(df.average_character_count), yaxis_index = 1)
#  .reversal_axis()
 .set_series_opts(label_opts=opts.LabelOpts(position="top", rotate= 30))
 .extend_axis(yaxis=opts.AxisOpts(name="Average Character Count", type_="value", position="left"))
#  .extend_axis(xaxis=opts.AxisOpts(name="Name", type_="value", position="left"), xaxis_opts )
#  .extend_axis(yaxis=opts.AxisOpts(type_="value", name="test2", position="left"))
 .set_global_opts(title_opts=opts.TitleOpts(title="Message Analytics for Elec Gang"),yaxis_opts=opts.AxisOpts(name="Message Count", position="right", offset=0), xaxis_opts=opts.AxisOpts(name="Name", position="bottom", offset=10, axislabel_opts = opts.LabelOpts(position="bottom", rotate= 30)))
)

bar.render_notebook()

## STATS FOR PARTICIPANT

In [6]:
for participant in participant_names:
    pdir = f'pictures/{participant}'
    if not path.exists(pdir):
        os.mkdir(pdir)
    ### reactions given to participant per person
    render_mpl_table(reactions_recieved_by_participant(total_messages, participant).reset_index())
    plt.title(f'Reactions {participant} has recieved')
    plt.savefig(f'{pdir}/reactions_recieved_by_participant {participant}')
    plt.close()

    ### reactions given by participant
    df = reactions_given_by_participant(total_messages, participant_names,participant)
    render_mpl_table(df.reset_index())
    plt.title(f'Reactions {participant} has given')
    plt.savefig(f'{pdir}/reactions_given_by_participant {participant}')
    plt.close()

    ### average reactions to participant message and max reactions
    # average_and_max_reactions_for_participant(total_messages, participant)

    ### self reacts
    df = pd.DataFrame(self_reacts(total_messages, participant))
    if len(df) > 0:
        render_mpl_table(df, col_width=9.0)
        plt.savefig(f'{pdir}/self_reacts_{participant}')
        plt.close()

    ### most common words by participant
    plot_dict(most_common_words_by_participant(total_messages, participant).head(20).to_dict());
    plt.savefig(f'{pdir}/most_common_words_{participant}.png')
    plt.close()

    ### usage per person
    usage_participant_highlighted(total_messages, participant_names, participant)
    plt.close()

In [171]:
check_reacts_and_translate = lambda m: {i['actor']:react_encoding_to_name[i['reaction']] for i in m['reactions']} if 'reactions' in m.keys() else {}

In [172]:
test = total_messages_with_id[70000]

In [173]:
most_angry = sorted(total_messages_with_id, key = lambda i: Counter(check_reacts_and_translate(total_messages_with_id[i]).values())['thinking'])[-3:]

In [176]:
for i in most_angry:
    message = copy.deepcopy(total_messages_with_id[i])
    message['reats_better'] = dict(Counter([react_encoding_to_name[i['reaction']] for i in message['reactions']]))
    del message['reactions']
    print(message, "\n")

{'sender_name': 'Michael Sprintson', 'timestamp_ms': 1634260937141, 'content': 'link me', 'type': 'Generic', 'is_unsent': False, 'reats_better': {'thinking': 5}} 

{'sender_name': 'Fredy Martinez', 'timestamp_ms': 1638252459639, 'content': 'Tastes like what I imagine pee taste likes', 'type': 'Generic', 'is_unsent': False, 'reats_better': {'thinking': 6, 'laughing': 2}} 

{'sender_name': 'Michael Sprintson', 'timestamp_ms': 1635643582201, 'content': 'nobodyâ\x80\x99s trying to do that unfortunately :(', 'type': 'Generic', 'is_unsent': False, 'reats_better': {'thinking': 8}} 



## meme chat

In [99]:
meme_total_messages, meme_total_messages_with_id, meme_participant_names = get_messages(messageloc = 'facebookdata/messages/inbox/hippoenthusiasts_-yoee2brvw/')
len(meme_total_messages)

17423

In [49]:
df = pd.Series(most_common_photo_sender(meme_total_messages)).to_frame()
df.columns = ['pcount']

In [50]:
react_dict = {}
for p in participant_names:
    participant_messages = [i for i in meme_total_messages if i['sender_name'] == p and 'photos' in i]
    reactions_recieved_by_participant = [i['reactions'] if 'reactions' in i.keys() else []for i in participant_messages]
    r = flat_list_n(reactions_recieved_by_participant)
    react_count = defaultdict(int, Counter([react_encoding_to_name[i['reaction']] for i in r]))
    react_dict[p] = react_count
react_df = pd.DataFrame(react_dict).T

In [51]:
gdf = react_df.join(df)

In [55]:
gdf.columns

Index(['cry', 'laughing', 'wow', 'heart', '<custom emote>', 'thinking',
       'thumbs up', 'thumbs down', 'angry', 'barf', 'heart eyes', 'teeth grit',
       'question mark', 'eyes', 'cowboy hat man', 'sweaty brow', 'pcount'],
      dtype='object')

In [64]:
gdf.sort_values(by = ['pcount'], inplace = True, ascending = False)

In [112]:
bar = (
 Bar(init_opts=opts.InitOpts(theme=ThemeType.DARK, width = "1100px", height = "550px"))
 .add_xaxis(list(gdf.index))
 .add_yaxis("Message Count", list(gdf.pcount), yaxis_index = 0)
#  .add_yaxis("cry Recieved", list(gdf.cry), yaxis_index = 1)
 .add_yaxis("wow Recieved", list(gdf.wow), yaxis_index = 1)
 .add_yaxis("heart Recieved", list(gdf.heart), yaxis_index = 1)
#  .add_yaxis("thumbs up Recieved", list(gdf["thumbs up"]), yaxis_index = 1)
#  .add_yaxis("thumbs down Recieved", list(gdf["thumbs down"]), yaxis_index = 1)
 .add_yaxis("angry Recieved", list(gdf.angry), yaxis_index = 1)
#  .add_yaxis("laughing Recieved", list(gdf.laughing), yaxis_index = 1)
#  .add_yaxis("heart eyes Recieved", list(gdf['heart eyes']), yaxis_index = 1)
#  .add_yaxis("teeth grit Recieved", list(gdf['teeth grit']), yaxis_index = 1)
#  .add_yaxis("question mark Recieved", list(gdf['question mark']), yaxis_index = 1)
#  .add_yaxis("eyes Recieved", list(gdf.eyes), yaxis_index = 1)
#  .add_yaxis("cowboy hat man Recieved", list(gdf['cowboy hat man']), yaxis_index = 1)
#  .add_yaxis("sweaty brow Recieved", list(gdf['sweaty brow']), yaxis_index = 1)
#  .reversal_axis()
 .set_series_opts(label_opts=opts.LabelOpts(position="top", rotate= 30))
 .extend_axis(yaxis=opts.AxisOpts(name="React Count", type_="value", position="left"))
#  .extend_axis(xaxis=opts.AxisOpts(name="Laughing Count", type_="value", position="left"))
#  .extend_axis(yaxis=opts.AxisOpts(type_="value", name="test2", position="left"))
 .set_global_opts(title_opts=opts.TitleOpts(title="Photo Analytics for Meme Chat"),yaxis_opts=opts.AxisOpts(name="Photo Count", position="right", offset=0), xaxis_opts=opts.AxisOpts(name="Name", position="bottom", offset=0, axislabel_opts = opts.LabelOpts(position="bottom", rotate= 30)))
)
bar.render_notebook()

In [116]:
(gdf['angry'] / gdf['pcount']).sort_values(ascending=False)

Fredy Martinez         0.200000
Kevin Tu               0.077739
Kevin Mejia            0.056497
Michael Sprintson      0.038576
Rene Carballo          0.036199
Feme Longe             0.034364
Justin Cheung          0.032787
Joshua Bae             0.021921
Zach Alvear            0.016818
Armando Amigon         0.016129
Patrick Chickey             NaN
Anirudh Kuchibhatla         NaN
dtype: float64

In [81]:
def most_reacted_to_message_list(total_messages_with_id):
    reacted_to_messages = {idx:i['reactions'] for idx, i in enumerate(total_messages_with_id) if 'reactions' in i.keys()}
    smes = sorted(reacted_to_messages, key = lambda i: len(reacted_to_messages[i]))
    maxid = smes[-1]
    reacted_to_message = total_messages_with_id[maxid]
    reacted_to_message['reats_better'] = dict(Counter([react_encoding_to_name[i['reaction']] for i in reacted_to_message['reactions']]))
    del reacted_to_message['reactions']
    return reacted_to_message, smes

In [85]:
_, s = most_reacted_to_message_list(meme_total_messages)

In [105]:
check_reacts = lambda m: {i['actor']:i['reaction'] for i in m['reactions']}

In [108]:
import copy

In [110]:
for i in s[-5:]:
    reacted_to_message = meme_total_messages[i]
    # print(reacted_to_message, "\n")
    reacted_to_message['reats_better'] = dict(Counter([react_encoding_to_name[j] for i,j in check_reacts(reacted_to_message).items()]))
    copy_reacted_to_message = copy.deepcopy(reacted_to_message)
    del copy_reacted_to_message['reactions']
    print(copy_reacted_to_message, "\n")


{'sender_name': 'Joshua Bae', 'timestamp_ms': 1636591161421, 'photos': [{'uri': 'messages/inbox/hippoenthusiasts_-yoee2brvw/photos/252124137_1509112756138300_7225121403099280047_n_3385386095065229.jpg', 'creation_timestamp': 1636591160}], 'type': 'Generic', 'is_unsent': False, 'reats_better': {'laughing': 5, 'wow': 2}} 

{'sender_name': 'Joshua Bae', 'timestamp_ms': 1638224598132, 'photos': [{'uri': 'messages/inbox/hippoenthusiasts_-yoee2brvw/photos/261920339_273175391519866_5705568004000864758_n_451490959816126.jpg', 'creation_timestamp': 1638224597}], 'type': 'Generic', 'is_unsent': False, 'reats_better': {'laughing': 7}} 

{'sender_name': 'Joshua Bae', 'timestamp_ms': 1638488731102, 'photos': [{'uri': 'messages/inbox/hippoenthusiasts_-yoee2brvw/photos/261064893_614693886402769_2984775687066124761_n_280071110601396.jpg', 'creation_timestamp': 1638488729}], 'type': 'Generic', 'is_unsent': False, 'reats_better': {'laughing': 7}} 

{'sender_name': 'Joshua Bae', 'timestamp_ms': 164083642

### reading level

In [ ]:
# rdng_levels = {}
# for pn in participant_names:
#     pmes = [i['content'] for i in total_messages if 'content' in i and pn == i['sender_name']]
#     textblob = reduce(lambda a,b: a + ' ' + b, pmes)
#     rdng_levels[pn] = textstat.smog_index(textblob)

In [ ]:
# pd.Series(rdng_levels).sort_values(ascending=False)

In [ ]:
# import textstat

In [ ]:
# textstat.gunning_fog('Esteemed friends and colleagues, please disregard this foolish momentary abandonment of my eloquence related endeavors. When faced with the visage of statistics and scholarship, feeling the exhilarating thrill of problem solving, I briefly forgot myself and the notions to which I had previously committed myself. As I mentioned before, the further we become removed from this discussion the more likely and frequent such lapses will become, but such is life.')

In [ ]:
# word_usage_highlighted(total_messages, participant_names, 'code')
# plt.close()